In [2]:
# Step 0: Install necessary libraries (if not already installed)
!pip -q install pandas numpy statsmodels patsy scipy openpyxl

# Step 1: Import all necessary libraries at the beginning
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, spearmanr, shapiro
from scipy.stats.mstats import winsorize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
import seaborn as sns

# Step 2: Define the data preparation function
# This function encapsulates data cleaning, type conversion, winsorization,
# log-transformation, and dummy variable creation.
def prepare_data_for_regression(df_raw):
    df = df_raw.copy()

    def as_num(s): return pd.to_numeric(s, errors='coerce')

    num_cols = [
        'Stock return 2024','Website index 2024','Momentum 12/2024',
        'Average Annual Total Return new','roa','Annual Dividend 2024',
        'Book to Market 2024','leverage 2024','Market cap 2024'
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = as_num(df[c])

    def winz_series(s: pd.Series, p=0.01):
        x = as_num(s).to_numpy(copy=True)
        w = winsorize(x, limits=[p, p])
        return pd.Series(w, index=s.index)

    vars_to_winz = [
        "Momentum 12/2024",
        "roa",
        "Annual Dividend 2024",
        "Book to Market 2024",
        "leverage 2024",
        "Average Annual Total Return new"
    ]
    for col in vars_to_winz:
        if col in df.columns:
            df[col + "_winz"] = winz_series(df[col], p=0.01)

    if 'Market cap 2024' in df.columns:
        df['log_mktcap'] = np.log(df['Market cap 2024'])

    if 'Industry Dummies ' in df.columns:
        df = pd.get_dummies(df, columns=['Industry Dummies '], drop_first=True)

    # The following block is important for the C(Industry) fixed effects model if it needs the original 'Industry' categorical column.
    # If 'Industry Dummies ' column from raw data was meant to become the categorical 'Industry' column,
    # the previous pd.get_dummies would have removed it. We ensure 'Industry' exists for fixed effects.
    # Assuming the original excel had a column 'Industry' in addition to 'Industry Dummies ' for proper fixed effects.
    if 'Industry' in df_raw.columns and 'Industry' not in df.columns:
        df['Industry'] = df_raw['Industry'].copy()
    if 'Industry' in df.columns:
        df['Industry'] = df['Industry'].astype('category').cat.add_categories(['Unknown']).fillna('Unknown')

    return df

# Step 3: Load the raw data and prepare it using the defined function
# This creates the df_prepared DataFrame which is used throughout the analysis.
print("\n--- Step 3: Loading and Preparing Data ---")
excel_file_path = '/content/experiment 2024.xlsx'
df_raw = pd.read_excel(excel_file_path)
df_prepared = prepare_data_for_regression(df_raw)
print("Head of prepared DataFrame:")
display(df_prepared.head())

# Step 4: Display descriptive statistics for relevant numerical columns
# This provides a summary of the central tendency, dispersion, and shape of the data.
print("\n--- Step 4: Descriptive Statistics ---")
relevant_cols_desc = ['log_mktcap', 'Annual Dividend 2024_winz', 'Beta 20242',
                      'Average Annual Total Return new_winz', 'Website index 2024',
                      'leverage 2024_winz', 'roa_winz', 'Momentum 12/2024_winz',
                      'Book to Market 2024_winz', 'Stock return 2024']
descriptive_stats = df_prepared[relevant_cols_desc].describe()
print("Descriptive Statistics for relevant columns:")
display(descriptive_stats)

# Step 5: Perform Outlier Analysis using IQR method and visualize with histograms
# This identifies and visualizes extreme values in key variables.
print("\n--- Step 5: Outlier Analysis ---")
id_col = 'Company'
outliers_dict = {}
outlier_names_by_col = {}
unique_outlier_names = set()

for col in relevant_cols_desc:
    s = df_prepared[col]
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    mask = (s < lower_bound) | (s > upper_bound)
    col_outliers = df_prepared.loc[mask]
    outlier_count = col_outliers.shape[0]
    outliers_dict[col] = outlier_count
    print(f"Outliers in {col}: {outlier_count}")

    if outlier_count > 0:
        names = col_outliers[id_col].astype(str).tolist()
        outlier_names_by_col[col] = names
        unique_outlier_names.update(names)
        print(f"  Outlier names for {col}: {names}")
        #display(col_outliers) # Uncomment to display outlier dataframes for each column

print("Unique outlier names across any metric:")
print(sorted(unique_outlier_names))

print("Generating outlier histograms...")
for col in relevant_cols_desc:
    s = df_prepared[col]
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    plt.figure(figsize=(10, 6))
    sns.histplot(data=df_prepared, x=col, kde=True)
    outliers_data = df_prepared.loc[(s < lower_bound) | (s > upper_bound)]
    plt.scatter(outliers_data[col], np.zeros(outliers_data.shape[0]), color='red', label='Outliers')
    plt.title(f'Distribution of {col} with Outliers')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
display(outliers_dict)

# Step 6: Calculate and visualize the Correlation Matrix
# This helps understand linear relationships between variables and check for multicollinearity.
print("\n--- Step 6: Correlation Matrix ---")
df_corr = df_prepared[relevant_cols_desc].dropna()
correlation_matrix = df_corr.corr()
print("Correlation Matrix:")
display(correlation_matrix)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Relevant Numerical Columns')
plt.show()

# Step 7: Calculate Variance Inflation Factors (VIFs) for independent variables
# This quantifies the severity of multicollinearity in an OLS regression.
print("\n--- Step 7: Variance Inflation Factors (VIFs) ---")
independent_vars_for_vif = [
    'log_mktcap',
    'Annual Dividend 2024_winz',
    'Beta 20242',
    'Average Annual Total Return new_winz',
    'Website index 2024',
    'leverage 2024_winz',
    'roa_winz',
    'Momentum 12/2024_winz',
    'Book to Market 2024_winz'
]
industry_dummy_cols = [col for col in df_prepared.columns if col.startswith('Industry Dummies _')]
independent_vars_for_vif.extend(industry_dummy_cols)

df_for_vif = df_prepared[independent_vars_for_vif].copy()
for col in industry_dummy_cols:
    if col in df_for_vif.columns and df_for_vif[col].dtype == 'bool':
        df_for_vif[col] = df_for_vif[col].astype(int)

df_for_vif = df_for_vif.dropna()
X_vif = sm.add_constant(df_for_vif)

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
print("VIF Values:")
display(vif_data)

# Step 8: Analyze the distribution of the dependent variable ('Stock return 2024')
# This checks for normality and other distributional properties.
print("\n--- Step 8: Dependent Variable Distribution ---")
plt.hist(df_prepared['Stock return 2024'].dropna(), bins=30, edgecolor='k')
plt.xlabel('Stock Return 2024')
plt.ylabel('Frequency')
plt.title('Distribution of 2024 Stock Returns')
plt.show()
print(f"Skewness: {skew(df_prepared['Stock return 2024'].dropna())}")
print(f"Kurtosis: {kurtosis(df_prepared['Stock return 2024'].dropna())}")

# Step 9: Preliminary OLS Regression & Residual Tests
# Fits a basic OLS model, checks for heteroscedasticity (Breusch-Pagan) and normality of residuals (Shapiro-Wilk).
print("\n--- Step 9: Preliminary OLS Regression & Residual Tests ---")
X_cols_prelim = [
    'log_mktcap', 'Annual Dividend 2024_winz', 'Beta 20242',
    'Average Annual Total Return new_winz', 'Website index 2024',
    'leverage 2024_winz', 'roa_winz', 'Momentum 12/2024_winz',
    'Book to Market 2024_winz'
]
industry_dummy_cols = [col for col in df_prepared.columns if col.startswith('Industry Dummies _')]
X_cols_prelim.extend(industry_dummy_cols)

X_model_prelim = df_prepared[X_cols_prelim].copy()
for col in industry_dummy_cols:
    if col in X_model_prelim.columns and X_model_prelim[col].dtype == 'bool':
        X_model_prelim[col] = X_model_prelim[col].astype(int)
X_model_prelim = X_model_prelim.dropna()
X_model_prelim_const = sm.add_constant(X_model_prelim)
y_model_prelim = df_prepared.loc[X_model_prelim_const.index, 'Stock return 2024']

model_prelim = sm.OLS(y_model_prelim, X_model_prelim_const).fit()
bp_test = het_breuschpagan(model_prelim.resid, model_prelim.model.exog)
print(f"Breusch-Pagan test p-value: {bp_test[1]}")

stat_shapiro, p_shapiro = shapiro(model_prelim.resid)
print(f"Shapiro-Wilk test p-value: {p_shapiro}")

# Step 10: Influence Diagnostics (Leverage and Cook's D)
# Identifies observations that have a disproportionate impact on the regression results.
print("\n--- Step 10: Influence Diagnostics ---")
influence_prelim = OLSInfluence(model_prelim)
leverage_prelim = influence_prelim.hat_matrix_diag
cooks_d_prelim = influence_prelim.cooks_distance[0]

n_prelim = len(X_model_prelim_const)
p_prelim = X_model_prelim_const.shape[1]
high_leverage_threshold = 2 * p_prelim / n_prelim
high_cooks_threshold = 4 / n_prelim

high_leverage_prelim = leverage_prelim > high_leverage_threshold
high_cooks_prelim = cooks_d_prelim > high_cooks_threshold

print(f"High leverage points (threshold {high_leverage_threshold:.4f}): {high_leverage_prelim.sum()}")
print(f"High Cook's D points (threshold {high_cooks_threshold:.4f}): {high_cooks_prelim.sum()}")

aligned_index_prelim = df_prepared.loc[X_model_prelim_const.index].index

diag_prelim = pd.DataFrame({
    'y': model_prelim.model.endog,
    'fitted': model_prelim.fittedvalues,
    'resid': model_prelim.resid,
    'std_resid': influence_prelim.resid_studentized_internal,
    'leverage': leverage_prelim,
    "cooks_d": cooks_d_prelim
}, index=aligned_index_prelim)

print("Top potentially influential points (by Cook's D from preliminary model):")
if 'Company' in df_prepared.columns:
    df_companies_prelim = df_prepared.loc[diag_prelim.index, ['Company']]
    print(df_companies_prelim.join(diag_prelim[high_leverage_prelim | high_cooks_prelim]).sort_values('cooks_d', ascending=False).head(15))
else:
    print(diag_prelim[high_leverage_prelim | high_cooks_prelim].sort_values('cooks_d', ascending=False).head(15))

# Step 11: Main OLS Regression with HC3 Robust Standard Errors
# This is a core regression model, reporting coefficients, standard errors, and significance.
print("\n--- Step 11: Main OLS Regression (HC3 Robust SEs) ---")
df_main = df_prepared.copy()
formula_main = """
Q('Stock return 2024') ~ Q('Website index 2024')
  + Q('Annual Dividend 2024_winz')
  + Q('Beta 20242')
  + Q('Average Annual Total Return new_winz')
  + Q('leverage 2024_winz')
  + Q('roa_winz')
  + Q('Momentum 12/2024_winz')
  + Q('Book to Market 2024_winz')
  + log_mktcap
"""
industry_dummy_cols_main = [col for col in df_main.columns if col.startswith('Industry Dummies _')]
if industry_dummy_cols_main:
    formula_main += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_main])

required_cols_main = [
    'Stock return 2024', 'Website index 2024', 'Annual Dividend 2024_winz',
    'Beta 20242', 'Average Annual Total Return new_winz', 'leverage 2024_winz',
    'roa_winz', 'Momentum 12/2024_winz', 'Book to Market 2024_winz', 'log_mktcap'
] + industry_dummy_cols_main

for col in required_cols_main:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')
for col in industry_dummy_cols_main:
    if col in df_main.columns and df_main[col].dtype == 'bool':
        df_main[col] = df_main[col].astype(int)

df_for_model_main = df_main.dropna(subset=required_cols_main).copy()
model_main_reg = smf.ols(formula=formula_main, data=df_for_model_main).fit(cov_type='HC3')
print("Summary of Main OLS Regression:")
print(model_main_reg.summary())

# Step 12: OLS Regression with Outlier Removal and Comparison
# This section demonstrates the impact of removing influential observations on model results.
print("\n--- Step 12: OLS Regression with Outlier Removal ---")
# Re-using formula_main and df_for_model_main from Step 11 for the base model
model_base_for_outlier_check = smf.ols(formula=formula_main, data=df_for_model_main).fit(cov_type='HC3')
print("\n=== Base model (with potential outliers) ===")
print(model_base_for_outlier_check.summary())

influence_base = OLSInfluence(model_base_for_outlier_check)
lev_base = influence_base.hat_matrix_diag
cd_base, _ = influence_base.cooks_distance
std_resid_base = influence_base.resid_studentized_internal

n_base = len(df_for_model_main)
p_base = model_base_for_outlier_check.df_model + 1

lev_thr_base = 2 * p_base / n_base
cd_thr_base = 4 / n_base

diag_base = pd.DataFrame({
    'leverage': lev_base,
    'cooks_d': cd_base,
    'y': model_base_for_outlier_check.model.endog,
    'fitted': model_base_for_outlier_check.fittedvalues,
    'resid': model_base_for_outlier_check.resid,
    'std_resid': std_resid_base
}, index=df_for_model_main.index)

diag_base['high_leverage'] = diag_base['leverage'] > lev_thr_base
diag_base['high_cooksd'] = diag_base['cooks_d'] > cd_thr_base
diag_base['high_abs_resid'] = diag_base['std_resid'].abs() > 3
diag_base['is_outlier'] = diag_base[['high_leverage','high_cooksd','high_abs_resid']].any(axis=1)

print("\nOutlier counts (rules triggered for base model):")
print(diag_base[['high_leverage','high_cooksd','high_abs_resid','is_outlier']].sum())

d_with_flags = df_for_model_main.join(diag_base[['high_leverage','high_cooksd','high_abs_resid','is_outlier']], how='left')
d_clean = d_with_flags[d_with_flags['is_outlier'] != True].copy()

# --- FIX for 'Industry Dummies _11' --- START
# Based on investigation, 'Industry Dummies _11' had only 1 observation in df_prepared
# and this observation was removed during outlier cleaning, leading to zero variance
# in d_clean. This causes perfect multicollinearity with the intercept term.
# We exclude it from the formula for the clean model.
industry_dummy_cols_fixed_for_clean_model = [col for col in industry_dummy_cols_main if col != 'Industry Dummies _11']

formula_clean_fixed = """
Q('Stock return 2024') ~ Q('Website index 2024')
  + Q('Annual Dividend 2024_winz')
  + Q('Beta 20242')
  + Q('Average Annual Total Return new_winz')
  + Q('leverage 2024_winz')
  + Q('roa_winz')
  + Q('Momentum 12/2024_winz')
  + Q('Book to Market 2024_winz')
  + log_mktcap
"""
if industry_dummy_cols_fixed_for_clean_model:
    formula_clean_fixed += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_fixed_for_clean_model])

model_clean = smf.ols(formula=formula_clean_fixed, data=d_clean).fit(cov_type='HC3')
# --- FIX for 'Industry Dummies _11' --- END

print("\n=== Clean model (outliers removed, Industry Dummies _11 excluded) ===")
print(model_clean.summary())

before_comp = pd.DataFrame({'coef': model_base_for_outlier_check.params, 'se': model_base_for_outlier_check.bse, 'pval': model_base_for_outlier_check.pvalues})
after_comp  = pd.DataFrame({'coef': model_clean.params, 'se': model_clean.bse, 'pval': model_clean.pvalues})
comp = before_comp.join(after_comp, lsuffix='_before', rsuffix='_after')
print("\n=== Coefficients before vs after outlier removal ===")
display(comp)

# Step 13: Bivariate OLS Regression
# A simple regression model to show the isolated effect of 'Website index 2024'.
print("\n--- Step 13: Bivariate OLS Regression ---")
df_biv = df_prepared.copy()
m_biv = smf.ols("Q('Stock return 2024') ~ Q('Website index 2024')", data=df_biv.dropna(subset=['Stock return 2024', 'Website index 2024'])).fit(cov_type='HC3')
print("Summary of Bivariate OLS Regression (Website Index):")
print(m_biv.summary())

# Step 14: Multivariate OLS Regression (Winsorized only)
# A regression using only winsorized and log-transformed variables.
# Modifying to include 'Beta 20242' for consistency with Main OLS model
print("\n--- Step 14: Multivariate OLS Regression (Winsorized - with Beta) ---")
df_mw = df_prepared.copy()
formula_mw = """
Q('Stock return 2024') ~ Q('Website index 2024')
 + Q('Momentum 12/2024_winz')
 + Q('Average Annual Total Return new_winz')
 + Q('roa_winz')
 + Q('Annual Dividend 2024_winz')
 + Q('Book to Market 2024_winz')
 + Q('leverage 2024_winz')
 + log_mktcap
 + Q('Beta 20242')
"""
industry_dummy_cols_mw = [col for col in df_mw.columns if col.startswith('Industry Dummies _')]
# Exclude 'Industry Dummies _11' from this model's dummies as well
industry_dummy_cols_mw = [col for col in industry_dummy_cols_mw if col != 'Industry Dummies _11']

if industry_dummy_cols_mw:
    formula_mw += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_mw])

needed_cols_mw = [
    'Stock return 2024','Website index 2024',
    'Momentum 12/2024_winz','Average Annual Total Return new_winz',
    'roa_winz','Annual Dividend 2024_winz','Book to Market 2024_winz',
    'leverage 2024_winz','log_mktcap', 'Beta 20242'
] + industry_dummy_cols_mw

for c in needed_cols_mw:
    df_mw[c] = pd.to_numeric(df_mw[c], errors='coerce')
for col in industry_dummy_cols_mw:
    if col in df_mw.columns and df_mw[col].dtype == 'bool':
        df_mw[col] = df_mw[col].astype(int)

d_w = df_mw.dropna(subset=needed_cols_mw).copy()
m_w = smf.ols(formula=formula_mw, data=d_w).fit(cov_type='HC3')
print("Summary of Multivariate OLS Regression (Winsorized - with Beta):")
print(m_w.summary())

# Step 15: Alternative OLS Model with Mixed Raw/Winsorized Variables
# A regression comparing the impact of using raw vs. winsorized variables for some predictors.
print("\n--- Step 15: OLS Regression (Mixed Raw/Winsorized) ---")
df_mixed = df_prepared.copy()
formula_mixed = """
Q('Stock return 2024') ~ Q('Website index 2024')
 + Q('Momentum 12/2024_winz')
 + Q('Average Annual Total Return new')
 + Q('roa_winz')
 + Q('Annual Dividend 2024')
 + Q('Book to Market 2024')
 + Q('leverage 2024')
 + log_mktcap
"""
industry_dummy_cols_mixed = [col for col in df_mixed.columns if col.startswith('Industry Dummies _')]
# Exclude 'Industry Dummies _11' from this model's dummies as well
industry_dummy_cols_mixed = [col for col in industry_dummy_cols_mixed if col != 'Industry Dummies _11']

if industry_dummy_cols_mixed:
    formula_mixed += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_mixed])

needed_cols_mixed = [
    'Stock return 2024','Website index 2024',
    'Momentum 12/2024_winz','Average Annual Total Return new',
    'roa_winz','Annual Dividend 2024','Book to Market 2024',
    'leverage 2024','log_mktcap'
] + industry_dummy_cols_mixed

for c in needed_cols_mixed:
    df_mixed[c] = pd.to_numeric(df_mixed[c], errors='coerce')
for col in industry_dummy_cols_mixed:
    if col in df_mixed.columns and df_mixed[col].dtype == 'bool':
        df_mixed[col] = df_mixed[col].astype(int)

d_mixed = df_mixed.dropna(subset=needed_cols_mixed).copy()
m_mixed = smf.ols(formula=formula_mixed, data=d_mixed).fit(cov_type='HC3')
print("Summary of OLS Regression (Mixed Raw/Winsorized):")
print(m_mixed.summary())

# Step 16: OLS Regression (Formula A - Comprehensive Winsorized)
# A specific OLS model combining several winsorized predictors and industry dummies.
print("\n--- Step 16: OLS Regression (Formula A) ---")
df_A = df_prepared.copy()
formula_A_comp = """
Q('Stock return 2024') ~ Q('Website index 2024')
 + Q('Momentum 12/2024_winz')
 + Q('roa_winz')
 + Q('Annual Dividend 2024_winz')
 + Q('Book to Market 2024_winz')
 + Q('leverage 2024_winz')
 + log_mktcap
"""
industry_dummy_cols_A = [col for col in df_A.columns if col.startswith('Industry Dummies _')]
# Exclude 'Industry Dummies _11' from this model's dummies as well
industry_dummy_cols_A = [col for col in industry_dummy_cols_A if col != 'Industry Dummies _11']

if industry_dummy_cols_A:
    formula_A_comp += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_A])

need_A_comp = ['Stock return 2024','Website index 2024','Momentum 12/2024_winz',
          'roa_winz','Annual Dividend 2024_winz','Book to Market 2024_winz',
          'leverage 2024_winz','log_mktcap'] + industry_dummy_cols_A

for c in need_A_comp:
    df_A[c] = pd.to_numeric(df_A[c], errors='coerce')
for col in industry_dummy_cols_A:
    if col in df_A.columns and df_A[col].dtype == 'bool':
        df_A[col] = df_A[col].astype(int)

dA_comp = df_A.dropna(subset=need_A_comp).copy()
mA_comp = smf.ols(formula=formula_A_comp, data=dA_comp).fit(cov_type='HC3')
print("Summary of OLS Regression (Formula A):")
print(mA_comp.summary())

# Step 17: OLS Regression (Formula B, Log-Y, Fixed Effects)
# Explores different model specifications: excluding momentum, log-transforming Y, and using fixed effects.
print("\n--- Step 17: OLS Regression (Formula B, Log-Y, Fixed Effects) ---")
df_spec = df_prepared.copy()

# Create a list of all existing industry dummy columns, excluding '_11' for consistency
all_industry_dummy_cols_in_df_prepared = [col for col in df_prepared.columns if col.startswith('Industry Dummies _')]
industry_dummy_cols_for_all_models = [col for col in all_industry_dummy_cols_in_df_prepared if col != 'Industry Dummies _11']

# Formula B: Excluding Momentum (winsorized)
formula_B_spec = """
Q('Stock return 2024') ~ Q('Website index 2024')
 + Q('Average Annual Total Return new_winz')
 + Q('roa_winz')
 + Q('Annual Dividend 2024_winz')
 + Q('Book to Market 2024_winz')
 + Q('leverage 2024_winz')
 + log_mktcap
"""
if industry_dummy_cols_for_all_models:
    formula_B_spec += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_for_all_models])

need_B_spec = ['Stock return 2024','Website index 2024','Average Annual Total Return new_winz',
          'roa_winz','Annual Dividend 2024_winz','Book to Market 2024_winz',
          'leverage 2024_winz','log_mktcap'] + industry_dummy_cols_for_all_models

for c in need_B_spec:
    df_spec[c] = pd.to_numeric(df_spec[c], errors='coerce')
for col in industry_dummy_cols_for_all_models:
    if col in df_spec.columns and df_spec[col].dtype == 'bool':
        df_spec[col] = df_spec[col].astype(int)

dB_spec = df_spec.dropna(subset=need_B_spec).copy()
mB_spec = smf.ols(formula=formula_B_spec, data=dB_spec).fit(cov_type='HC3')
print("\nSummary of OLS Regression (Formula B - No Momentum):")
print(mB_spec.summary())

# Log-transform dependent variable
df_spec['log1p_return_2024'] = np.log1p(df_spec['Stock return 2024'])

# Formula A with log-transformed dependent variable
formula_A_logy_spec = """
log1p_return_2024 ~ Q('Website index 2024')
 + Q('Momentum 12/2024_winz')
 + Q('roa_winz')
 + Q('Annual Dividend 2024_winz')
 + Q('Book to Market 2024_winz')
 + Q('leverage 2024_winz')
 + log_mktcap
"""
if industry_dummy_cols_for_all_models:
    formula_A_logy_spec += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_for_all_models])

need_A_logy_vars_spec = ['Website index 2024','Momentum 12/2024_winz',
                 'roa_winz','Annual Dividend 2024_winz','Book to Market 2024_winz',
                 'leverage 2024_winz','log_mktcap'] + industry_dummy_cols_for_all_models

need_A_logy_spec = ['log1p_return_2024'] + need_A_logy_vars_spec

for c in need_A_logy_spec:
    df_spec[c] = pd.to_numeric(df_spec[c], errors='coerce')
for col in industry_dummy_cols_for_all_models:
    if col in df_spec.columns and df_spec[col].dtype == 'bool':
        df_spec[col] = df_spec[col].astype(int)

dA_logy_spec = df_spec.dropna(subset=need_A_logy_spec).copy()
mA_logy_spec = smf.ols(formula=formula_A_logy_spec, data=dA_logy_spec).fit(cov_type='HC3')
print("\nSummary of OLS Regression (Formula A with Log-Transformed Y):")
print(mA_logy_spec.summary())

# Formula A with fixed effects for Industry (using explicit dummy variables)
# The previous implementation using C(Industry) caused extreme multicollinearity
# because of potential conflicts with other variables or sparse categories after dropna.
# We now use the explicit Industry Dummies generated earlier, excluding the problematic _11.
print("\nSummary of OLS Regression (Formula A with Industry Dummies - Clarified Fixed Effects):")

formula_A_fe_spec_fixed = """
Q('Stock return 2024') ~ Q('Website index 2024')
 + Q('Momentum 12/2024_winz')
 + Q('roa_winz')
 + Q('Annual Dividend 2024_winz')
 + Q('Book to Market 2024_winz')
 + Q('leverage 2024_winz')
 + log_mktcap
"""
if industry_dummy_cols_for_all_models:
    formula_A_fe_spec_fixed += " + " + " + ".join([f"Q('{col}')" for col in industry_dummy_cols_for_all_models])

need_A_fe_spec_fixed = ['Stock return 2024','Website index 2024','Momentum 12/2024_winz',
          'roa_winz','Annual Dividend 2024_winz','Book to Market 2024_winz',
          'leverage 2024_winz','log_mktcap'] + industry_dummy_cols_for_all_models

for c in need_A_fe_spec_fixed:
    df_spec[c] = pd.to_numeric(df_spec[c], errors='coerce')

# Ensure dummy columns are int (if not already)
for col in industry_dummy_cols_for_all_models:
    if col in df_spec.columns and df_spec[col].dtype == 'bool':
        df_spec[col] = df_spec[col].astype(int)

dA_fe_spec_fixed = df_spec.dropna(subset=need_A_fe_spec_fixed).copy()
mA_fe_spec_fixed = smf.ols(formula=formula_A_fe_spec_fixed, data=dA_fe_spec_fixed).fit(cov_type='HC3')
print(mA_fe_spec_fixed.summary())

# Step 18: VIF Calculation for Formula A's Independent Variables
# Re-calculates VIFs for the specific set of independent variables in Formula A.
print("\n--- Step 18: VIF for Formula A Independent Variables ---")
independent_vars_mA_vif = [
    "Website index 2024",
    "Momentum 12/2024_winz",
    "roa_winz",
    "Annual Dividend 2024_winz",
    "Book to Market 2024_winz",
    "leverage 2024_winz",
    "log_mktcap"
]
# Use the same filtered list of industry dummies for VIF calculation
industry_dummy_cols_mA_vif = [col for col in dA_comp.columns if col.startswith('Industry Dummies _') and col != 'Industry Dummies _11']
independent_vars_mA_vif.extend(industry_dummy_cols_mA_vif)

X_A_vif = dA_comp[independent_vars_mA_vif].copy()
for col in industry_dummy_cols_mA_vif:
    if col in X_A_vif.columns and X_A_vif[col].dtype == 'bool':
        X_A_vif[col] = X_A_vif[col].astype(int)
X_A_vif = X_A_vif.dropna()
X_A_vif_const = sm.add_constant(X_A_vif, has_constant='add')

vif_df_A = pd.DataFrame({
    'variable': X_A_vif_const.columns,
    'VIF': [variance_inflation_factor(X_A_vif_const.values, i) for i in range(X_A_vif_const.shape[1])]
})
print("VIF Values for Formula A:")
display(vif_df_A)

# Step 19: Rank Regression Analysis
# Explores relationships using rank-transformed variables, robust to non-normality.
print("\n--- Step 19: Rank Regression Analysis ---")
df_rank = df_prepared.copy()

df_rank['rank_return'] = df_rank['Stock return 2024'].rank(pct=True)
df_rank['rank_website'] = df_rank['Website index 2024'].rank(pct=True)
df_rank['rank_mom'] = df_rank['Momentum 12/2024_winz'].rank(pct=True)
df_rank['rank_roa'] = df_rank['roa_winz'].rank(pct=True)
df_rank['rank_div'] = df_rank['Annual Dividend 2024_winz'].rank(pct=True)
df_rank['rank_btm'] = df_rank['Book to Market 2024_winz'].rank(pct=True)
df_rank['rank_logmktcap'] = df_rank['log_mktcap'].rank(pct=True)

# Spearman (unconditional) correlation
mask_spearman = df_rank['Website index 2024'].notna() & df_rank['Stock return 2024'].notna()
rho_spearman, p_spearman = spearmanr(df_rank.loc[mask_spearman, 'Website index 2024'], df_rank.loc[mask_spearman, 'Stock return 2024'])
print(f"Spearman rho(Website, Return) = {rho_spearman:.3f}, p = {p_spearman:.4f}")

# Bivariate rank regression
print("\nSummary of Bivariate Rank Regression:")
m_rank_biv = smf.ols("rank_return ~ rank_website", data=df_rank.dropna(subset=['rank_return','rank_website'])).fit(cov_type='HC3')
print(m_rank_biv.summary())

# Multivariate rank regression
print("\nSummary of Multivariate Rank Regression:")
need_rank_mv = ['rank_return','rank_website','rank_mom','rank_roa','rank_div','rank_btm','rank_logmktcap']
d_rank_mv = df_rank.dropna(subset=need_rank_mv).copy()
m_rank_mv = smf.ols("rank_return ~ rank_website + rank_mom + rank_roa + rank_div + rank_btm + rank_logmktcap", data=d_rank_mv).fit(cov_type='HC3')
print(m_rank_mv.summary())


--- Step 3: Loading and Preparing Data ---


FileNotFoundError: [Errno 2] No such file or directory: '/content/experiment 2024.xlsx'